In [ ]:
# Import from refactored modules
import torch
import numpy as np
import os
import time
from torch.utils.data import DataLoader

# Import from new modular structure
from data_generation import GaussianMixtureModel, generate_icl_gmm_data, generate_iwl_gmm_data
from datasets import ICLGMMDataset, collate_fn
from models import MatrixTreeMarkovICL, RandomPolynomialICL, NonlinearMarkovICL
from training import train_model
from evaluation import test_icl
from config import ExperimentConfig

# ============================================================
# Data Generation Parameters
# ============================================================
L = 256                      # Number of output classes
K = 256                      # Number of GMM classes for data generation
D = 2                        # Dimension of input features
N = 3                        # Number of context examples per task
B = 1                        # Burstiness parameter (zipfian sampling weight)
epsilon = 1e-3               # Within-class noise (standard deviation)
seed = 20                    # Random seed for reproducibility
exact_copy = True            # If True, query is exact copy of a context item
shuffle_context = True       # Whether to shuffle context order during training
offset = 0.0                 # Offset applied to GMM centers
min_max_choice = None        # Optional constraint on min/max class indices
unique_labels = False        # If True, ensure all context labels are unique

# ============================================================
# Model Architecture Parameters
# ============================================================
n_nodes = 6                  # Number of nodes in the Markov chain
transform_func = 'exp'       # Transformation function: 'exp', 'relu', or 'elu'
learn_base_rates = False     # If True, allow gradient updates to unmasked base rates

# ============================================================
# Sparsity Parameters - K_params (context-dependent modulation)
# ============================================================
sparsity_rho_edge_K = 0.2    # Fraction of (i,j) edges with K parameters (0.0 = all masked)
sparsity_rho_all_K = 0.2     # Fraction of individual K parameters to keep (0.0 = all masked)

# ============================================================
# Sparsity Parameters - L_params (nonlinear interactions)
# ============================================================
sparsity_rho_edge_L = 0.0    # Fraction of (i,j,k) triplets with L parameters
sparsity_rho_all_L = 0.0    # Fraction of individual L parameters to keep

# ============================================================
# Sparsity Parameters - Base Rates
# ============================================================
sparsity_rho_edge_base_W = 1.0   # Fraction of (i,j) edges with base rates in W
sparsity_rho_edge_base_Y = 0.0   # Fraction of (i,j,k) triplets with base rates in Y
base_mask_value = float('-inf')            # Value for masked base rates: 0.0 (no bias) or float('-inf') (disable edge)

# ============================================================
# Training Parameters
# ============================================================
epochs = 30                  # Number of training epochs
lr = 0.0025                  # Learning rate
batch_size = 64              # Batch size for training
train_samples = 25000        # Number of training samples
val_samples = 2000           # Number of validation samples

# ============================================================
# Inference Parameters
# ============================================================
method = 'newton'            # Steady-state solver: 'newton', 'direct_solve', 'matrix_tree', 'linear_solver'
temperature = 1.0            # Softmax temperature for attention

# ============================================================
# Combined Parameter Dictionary
# ============================================================
params = {
    # Data generation
    'K': K,
    'L': L,
    'D': D,
    'N': N,
    'B': B,
    'epsilon': epsilon,
    'seed': seed,
    'exact_copy': exact_copy,
    'shuffle_context': shuffle_context,
    'offset': offset,
    'min_max_choice': min_max_choice,
    'unique_labels': unique_labels,
    
    # Model architecture
    'n_nodes': n_nodes,
    'transform_func': transform_func,
    'learn_base_rates': learn_base_rates,
            
    # Sparsity - K_params
    'sparsity_rho_edge_K': sparsity_rho_edge_K,
    'sparsity_rho_all_K': sparsity_rho_all_K,
    
    # Sparsity - L_params
    'sparsity_rho_edge_L': sparsity_rho_edge_L,
    'sparsity_rho_all_L': sparsity_rho_all_L,
    
    # Sparsity - Base rates
    'sparsity_rho_edge_base_W': sparsity_rho_edge_base_W,
    'sparsity_rho_edge_base_Y': sparsity_rho_edge_base_Y,
    'base_mask_value': base_mask_value,
    
    # Training
    'epochs': epochs,
    'lr': lr,
    'batch_size': batch_size,
    'train_samples': train_samples,
    'val_samples': val_samples,
    
    # Inference
    'method': method,
    'temperature': temperature
}


print("="*70)
print("MARKOV ICL - CLASSIFICATION (Softmax Output)")
print("="*70)
print(f"K={params['K']}, D={params['D']}, N={params['N']}, B={params['B']}, nodes={params['n_nodes']}")
print(f"Method: {params['method']}, Temperature: {params['temperature']}")
print("="*70)

# Set random seeds
torch.manual_seed(params['seed'])
np.random.seed(params['seed'])

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}\n")

# Create GMM with discrete labels (1 to L)
print("Creating GMM with discrete labels...")
gmm = GaussianMixtureModel(K=params['K'], D=params['D'], L=params['L'], epsilon=params['epsilon'], seed=params['seed'], offset=params['offset'])
print(f"  GMM: {params['K']} classes with labels randomly assigned from {{1, ..., {params['L']}}}")
print(f"  First 10 class labels: {gmm.class_to_label[:min(10, params['K'])].numpy()}")

# Generate data
print("\nGenerating data...")
train_data = generate_icl_gmm_data(gmm, params['train_samples'], params['N'], 
                                   novel_classes=False, exact_copy=params['exact_copy'], 
                                   B=params['B'], L=params['L'], shuffle_context=params['shuffle_context'], min_max_choice=params['min_max_choice'], unique_labels = params['unique_labels'])
val_data = generate_icl_gmm_data(gmm, params['val_samples'], params['N'], 
                                 novel_classes=False, exact_copy=params['exact_copy'], 
                                 B=params['B'], L=params['L'], shuffle_context=params['shuffle_context'], min_max_choice=params['min_max_choice'], unique_labels = params['unique_labels'])

train_loader = DataLoader(ICLGMMDataset(train_data), batch_size=params['batch_size'],
                          shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(ICLGMMDataset(val_data), batch_size=params['batch_size'],
                       collate_fn=collate_fn)

# Create model
print("\nCreating model...")
model = NonlinearMarkovICL(n_nodes=params['n_nodes'], z_dim=params['D'], 
                           L=params['L'], N=params['N'], 
                           learn_base_rates=params['learn_base_rates'], 
                           transform_func=params['transform_func'],
                           sparsity_rho_edge_K=params['sparsity_rho_edge_K'], 
                           sparsity_rho_all_K=params['sparsity_rho_all_K'],
                           sparsity_rho_edge_L=params['sparsity_rho_edge_L'], 
                           sparsity_rho_all_L=params['sparsity_rho_all_L'],
                           sparsity_rho_edge_base_W=params['sparsity_rho_edge_base_W'],
                           sparsity_rho_edge_base_Y=params['sparsity_rho_edge_base_Y'],
                           base_mask_value=params['base_mask_value'])

# Train with ICL/IWL tracking
start_time = time.time()
print("\nTraining...")
print("="*70)
history = train_model(model, train_loader, val_loader, device, 
                     n_epochs=params['epochs'], lr=params['lr'], 
                     method=params['method'], temperature=params['temperature'],
                     gmm=gmm, N=params['N'], B=params['B'], 
                     L=params['L'], exact_copy=params['exact_copy'],
                     eval_frequency=1, n_eval_samples=500, min_max_choice=params['min_max_choice'], unique_labels = params['unique_labels'])
                     
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

# Test
results = test_icl(model, gmm, params['N'], device, n_samples=1000, 
                  exact_copy=params['exact_copy'], B=params['B'], 
                  method=params['method'], L=params['L'],
                  temperature=params['temperature'], shuffle_context=params['shuffle_context'], min_max_choice=params['min_max_choice'], unique_labels = params['unique_labels'])

# # Save model
# os.makedirs('results', exist_ok=True)
# model_path = f'results/markov_icl_gmm_K{config.K}_N{config.N}_classification_T{config.temperature:.1f}.pt'
# torch.save(model.state_dict(), model_path)
# print(f"\n✓ Saved: {model_path}")

# Plot training history with ICL/IWL metrics
try:
    import matplotlib.pyplot as plt
    
    # Extract ICL/IWL metrics (filter out None values)
    epochs_eval = [i+1 for i, val in enumerate(history['iwl_acc']) if val is not None]
    iwl_acc = [val for val in history['iwl_acc'] if val is not None]
    icl_acc = [val for val in history['icl_acc'] if val is not None]
    #icl_swap_acc = [val for val in history['icl_swap_acc'] if val is not None]
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # Plot 1: Training/Val Accuracy
    ax1 = axes[0, 0]
    ax1.plot(history['train_acc'], label='Train Accuracy', alpha=0.7)
    ax1.plot(history['val_acc'], label='Val Accuracy', alpha=0.7)
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy (%)')
    ax1.set_title('Training Progress')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim([0, 100])
    
    # Plot 2: ICL vs IWL over time
    ax2 = axes[0, 1]
    if len(iwl_acc) > 0:
        ax2.plot(epochs_eval, iwl_acc, 'o-', label='IWL (In-Weight)', linewidth=2, markersize=6)
        ax2.plot(epochs_eval, icl_acc, 's-', label='ICL (In-Context)', linewidth=2, markersize=6)
        #ax2.plot(epochs_eval, icl_swap_acc, '^-', label='ICL Label Swap', linewidth=2, markersize=6)
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Accuracy (%)')
        ax2.set_title('ICL vs IWL During Training')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        ax2.set_ylim([0, 105])
    else:
        ax2.text(0.5, 0.5, 'No ICL/IWL data\n(set gmm parameter)', 
                ha='center', va='center', transform=ax2.transAxes)
    
    # Plot 3: Final test results
    ax3 = axes[1, 0]
    ax3.bar(['In-Distribution', 'Novel Classes'], 
            [results['in_dist'], results['novel_classes']],
            color=['skyblue', 'coral'])
    ax3.set_ylabel('Accuracy (%)')
    ax3.set_title('Final Test Results')
    ax3.axhline(y=80, color='g', linestyle='--', label='Good threshold (80%)', alpha=0.5)
    ax3.legend()
    ax3.grid(True, alpha=0.3, axis='y')
    ax3.set_ylim([0, 100])
    
    # Plot 4: Training/Val Loss
    ax4 = axes[1, 1]
    ax4.plot(history['train_loss'], label='Train Loss', alpha=0.7)
    ax4.plot(history['val_loss'], label='Val Loss', alpha=0.7)
    ax4.set_xlabel('Epoch')
    ax4.set_ylabel('Loss')
    ax4.set_title('Training Loss')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    if len(iwl_acc) > 0:
        print("\n" + "="*70)
        print("ICL/IWL SUMMARY")
        print("="*70)
        print(f"Final IWL Accuracy:        {iwl_acc[-1]:.2f}%")
        print(f"Final ICL Accuracy:  {icl_acc[-1]:.2f}%")
        #print(f"Final ICL Swap Accuracy:   {icl_swap_acc[-1]:.2f}%")
        print("="*70)
    
except ImportError:
    print("\n(Install matplotlib to see training plots)")

MARKOV ICL - CLASSIFICATION (Softmax Output)
K=256, D=2, N=3, B=1, nodes=6
Method: newton, Temperature: 1.0
Device: cpu

Creating GMM with discrete labels...
  GMM: 256 classes with labels randomly assigned from {1, ..., 256}
  First 10 class labels: [249. 186. 119. 246. 114. 171. 109.   9. 190. 144.]

Generating data...

Creating model...
  Initialized Nonlinear Markov ICL model (L=256 classes, attention over 3 context items)
  Nonlinear dynamics: W p + p Y p = 0
  Label modulation: False
  Base rates learnable: False
  Base mask value: -inf
  Sparsity K: rho_edge=0.200, rho_all=0.200
  Sparsity L: rho_edge=0.000, rho_all=0.000
  Sparsity base_W: rho_edge=1.000
  Sparsity base_Y: rho_edge=0.000
  K_params sparsity: 0.976 (7/288 active)
  L_params sparsity: 1.000 (0/1728 active)
  base_W sparsity: 0.000 (36/36 active)
  base_Y sparsity: 1.000 (0/216 active)
  Parameters: 2,034

Training...
Epoch  10 | Train: 38.07% | Val: 42.05% | IWL: 1.60% | ICL: 37.60%
Epoch  20 | Train: 41.72% | Va

In [1]:
# Get a single example from the training dataset
z_seq, labels, targets = train_loader.dataset[2000]

# Add batch dimension and move to device
z_seq = z_seq.unsqueeze(0).to(device)  # Shape: (1, N+1, D)
labels = labels.unsqueeze(0).to(device)  # Shape: (1, N)
targets = targets.unsqueeze(0).to(device).long() - 1  # Shape: (1,)

print("Example input:")
print(f"z_seq shape: {z_seq.shape}")
print(f"labels shape: {labels.shape}")
print(f"z_seq:\n{z_seq}")
print(f"labels: {labels}")

# Flatten z_seq as the model expects
z_flat = z_seq.reshape(1, -1)  # Shape: (1, (N+1)*D)
print(f"\nFlattened z shape: {z_flat.shape}")

# Compute the rate matrix W and rate tensor Y
model.eval()
with torch.no_grad():
    W_mat = model.compute_rate_matrix_W(z_flat)
    Y_tensor = model.compute_rate_tensor_Y(z_flat)

print(f"\nRate matrix W shape: {W_mat.shape}")
print(f"Rate matrix W:\n{W_mat[0]}")  # Print the first (and only) matrix in the batch

# Verify that columns sum to approximately zero (as they should for a rate matrix)
col_sums = W_mat[0].sum(dim=0)
print(f"\nColumn sums of W (should be ~0): {col_sums}")

print(f"\nRate tensor Y shape: {Y_tensor.shape}")
print(f"Rate tensor Y[0] (first batch):\n{Y_tensor[0]}")

# Verify the constraint: Y[j,j,k] = -sum_{i≠j} Y[i,j,k]
# Check for j=0, k=0 as an example
j, k = 0, 0
sum_off_diag = Y_tensor[0, :, j, k].sum() - Y_tensor[0, j, j, k]  # Sum excluding diagonal
print(f"\nConstraint check for Y[j={j},j={j},k={k}]:")
print(f"  Y[{j},{j},{k}] = {Y_tensor[0, j, j, k].item():.6f}")
print(f"  -Sum_{{i≠{j}}} Y[i,{j},{k}] = {-sum_off_diag.item():.6f}")
print(f"  Difference (should be ~0): {(Y_tensor[0, j, j, k] + sum_off_diag).item():.6e}")

# Compute steady state using nonlinear solver
n_iter = 50
step_size = 0.1

with torch.no_grad():
    p_steady = model.direct_solve_steady_state(W_mat, Y_tensor, n_iter=n_iter, step_size=step_size)

print(f"\nSteady state distribution (π):")
print(f"Shape: {p_steady.shape}")
print(f"π = {p_steady[0]}")
print(f"Sum of π (should be 1.0): {p_steady[0].sum().item()}")

# Verify the steady state: W p + p Y p should be approximately 0
with torch.no_grad():
    # Linear term: W @ p
    linear_term = torch.einsum("bij,bj->bi", W_mat, p_steady)
    
    # Quadratic term: sum_{j,k} Y[i,j,k] * p_j * p_k
    outer = p_steady.unsqueeze(2) * p_steady.unsqueeze(1)  # (batch_size, n, n)
    quadratic_term = torch.einsum("bijk,bjk->bi", Y_tensor, outer)
    
    # Total should be ~0
    total_drift = linear_term + quadratic_term
    
print(f"\nSteady state verification (W p + p Y p):")
print(f"Linear term (W @ p): {linear_term[0]}")
print(f"Quadratic term (p Y p): {quadratic_term[0]}")
print(f"Total (should be ~0): {total_drift[0]}")
print(f"Max absolute drift: {torch.abs(total_drift[0]).max().item():.6e}")

# Compute context position scores: q_m = Σ_k B_{k,m} * π_k
with torch.no_grad():
    q = torch.matmul(p_steady, model.B)  # (batch_size, N)
    attention = torch.softmax(q / temperature, dim=1)

print(f"\nContext position scores (q):")
print(f"q = {q[0]}")
print(f"\nAttention weights over context positions:")
print(f"attention = {attention[0]}")
print(f"Sum of attention (should be 1.0): {attention[0].sum().item()}")

# Optional: Also compute the full forward pass to see final output
with torch.no_grad():
    logits = model(z_seq, labels, method='direct_solve', temperature=temperature, 
                   n_iter=n_iter, step_size=step_size)
    probs = torch.exp(logits)  # Convert log-probs to probs
    predicted_class = logits.argmax(dim=1) + 1  # Convert back to 1-indexed
    
print(f"\nModel output:")
print(f"Logits shape: {logits.shape}")
print(f"Top 5 logits: {logits[0].topk(5)}")
print(f"Top 5 probabilities: {probs[0].topk(5)}")
print(f"Predicted class: {predicted_class.item()}")
print(f"True class: {targets.item() + 1}")
print(f"Correct: {predicted_class.item() == targets.item() + 1}")

NameError: name 'train_loader' is not defined

In [20]:
def direct_solve_steady_state_ext(model, W_batch, Y_batch, n_iter=50, step_size=0.05, eps=1e-12):
        """
        Solve nonlinear steady state using fixed-point iteration (gradient descent).
        
        Solves: W p + p Y p = 0 with sum(p) = 1 and p >= 0
        where (p Y p)_i = sum_{j,k} Y[i,j,k] p_j p_k
        
        Uses gradient descent to minimize ||W p + p Y p||, projecting onto the probability simplex.
        
        Args:
            W_batch: (batch_size, n_nodes, n_nodes) - linear rate matrix
            Y_batch: (batch_size, n_nodes, n_nodes, n_nodes) - nonlinear rate tensor
            n_iter: Number of fixed-point iterations (default: 50)
            step_size: Step size for gradient descent (default: 0.1)
            eps: Minimum value for clamping (default: 1e-12)
            
        Returns:
            p_batch: (batch_size, n_nodes) - steady state distributions
        """
        batch_size, n = W_batch.shape[0], model.n_nodes
        device = W_batch.device
        
        # Initialize from uniform distribution
        p_batch = torch.full((batch_size, n), 1.0/n, device=device)
        
        for _ in range(n_iter):
            # Compute quadratic term: Q_i = sum_{j,k} Y[i,j,k] p_j p_k
            # Outer product: p_j * p_k
            outer = p_batch.unsqueeze(2) * p_batch.unsqueeze(1)  # (batch_size, n, n)
            Q = torch.einsum("bijk,bjk->bi", Y_batch, outer)  # (batch_size, n)
            
            # Compute linear term: L_i = sum_j W[i,j] p_j
            L = torch.einsum("bij,bj->bi", W_batch, p_batch)  # (batch_size, n)
            
            # Total drift function F = W p + p Y p
            F = L + Q  # (batch_size, n)
            
            # Gradient descent step toward F(p) = 0
            p_batch = p_batch + step_size * F
            
            # Project onto probability simplex: clamp and normalize
            p_batch = torch.clamp(p_batch, min=eps)
            p_batch = p_batch / p_batch.sum(dim=1, keepdim=True)
        
        # Handle NaN/Inf (fallback to uniform)
        mask = torch.isnan(p_batch).any(dim=1) | torch.isinf(p_batch).any(dim=1)
        if mask.any():
            p_batch[mask] = 1.0 / n
        
        return p_batch


def direct_solve_steady_state_ext_mom(model, W_batch, Y_batch, n_iter=50, step_size=0.05, eps=1e-12, tol=1e-6, momentum=0.3):
        """
        Solve nonlinear steady state using fixed-point iteration (gradient descent).
        
        Solves: W p + p Y p = 0 with sum(p) = 1 and p >= 0
        where (p Y p)_i = sum_{j,k} Y[i,j,k] p_j p_k
        
        Uses gradient descent to minimize ||W p + p Y p||, projecting onto the probability simplex.
        
        Args:
            W_batch: (batch_size, n_nodes, n_nodes) - linear rate matrix
            Y_batch: (batch_size, n_nodes, n_nodes, n_nodes) - nonlinear rate tensor
            n_iter: Number of fixed-point iterations (default: 50)
            step_size: Step size for gradient descent (default: 0.1)
            eps: Minimum value for clamping (default: 1e-12)
            
        Returns:
            p_batch: (batch_size, n_nodes) - steady state distributions
        """
        batch_size, n = W_batch.shape[0], model.n_nodes
        device = W_batch.device
        
        # Initialize from uniform distribution
        p_batch = torch.full((batch_size, n), 1.0/n, device=device)
        
        # Momentum buffer
        velocity = torch.zeros_like(p_batch)
        
        for iter_idx in range(n_iter):
            # Compute quadratic term: Q_i = sum_{j,k} Y[i,j,k] p_j p_k
            outer = p_batch.unsqueeze(2) * p_batch.unsqueeze(1)  # (batch_size, n, n)
            Q = torch.einsum("bijk,bjk->bi", Y_batch, outer)  # (batch_size, n)
            
            # Compute linear term: L_i = sum_j W[i,j] p_j
            # Using bmm is slightly faster than einsum for matrix-vector multiply
            L = torch.bmm(W_batch, p_batch.unsqueeze(-1)).squeeze(-1)  # (batch_size, n)
            
            # Total drift function F = W p + p Y p
            F = L + Q  # (batch_size, n)
            
            # Check for early convergence (optional)
            max_drift = torch.abs(F).max().item()
            if max_drift < tol:
                break
            
            # Momentum update
            velocity = momentum * velocity + step_size * F
            p_batch = p_batch + velocity
            
            # Project onto probability simplex: clamp and normalize
            p_batch = torch.clamp(p_batch, min=eps)
            p_batch = p_batch / p_batch.sum(dim=1, keepdim=True)
        
        # Handle NaN/Inf (fallback to uniform)
        mask = torch.isnan(p_batch).any(dim=1) | torch.isinf(p_batch).any(dim=1)
        if mask.any():
            p_batch[mask] = 1.0 / n
        
        return p_batch

def newton_steady_state(model, W_batch, Y_batch, n_iter=10, eps=1e-12, tol=1e-8):
    """
    Newton's method for solving W p + p Y p = 0.
    
    Much faster convergence (typically 5-10 iterations vs 50+)
    """
    batch_size, n = W_batch.shape[0], model.n_nodes
    device = W_batch.device
    
    # Initialize from uniform distribution
    p_batch = torch.full((batch_size, n), 1.0/n, device=device)
    
    for _ in range(n_iter):
        # Compute F(p) = W p + p Y p
        outer = p_batch.unsqueeze(2) * p_batch.unsqueeze(1)
        Q = torch.einsum("bijk,bjk->bi", Y_batch, outer)
        L = torch.bmm(W_batch, p_batch.unsqueeze(-1)).squeeze(-1)
        F = L + Q
        
        # Check convergence
        if torch.abs(F).max() < tol:
            break
        
        # Compute Jacobian: J[i,j] = dF_i/dp_j
        # J = W + (Y[:,:,j,k] + Y[:,i,k,j]) * p_k (summed over k)
        # For quadratic term: d/dp_j (sum_k,l Y[i,k,l] p_k p_l) = sum_l Y[i,j,l]*p_l + sum_k Y[i,k,j]*p_k
        
        # Simplified: J = W + 2 * (Y contracted with p on last index)
        Y_p_contract = torch.einsum("bijk,bk->bij", Y_batch, p_batch)  # (b, n, n)
        Y_p_contract_T = torch.einsum("bijk,bj->bik", Y_batch, p_batch)  # (b, n, n)
        J = W_batch + Y_p_contract + Y_p_contract_T  # (batch, n, n)
        
        # Add constraint: last row enforces sum(p) = 1
        J_constrained = J.clone()
        J_constrained[:, -1, :] = 1.0
        
        # RHS
        b = -F
        b[:, -1] = 1.0 - p_batch.sum(dim=1)  # Enforce sum constraint
        
        # Solve J * delta_p = -F
        delta_p = torch.linalg.solve(J_constrained, b)
        
        # Update with line search damping
        alpha = 1.0
        p_new = p_batch + alpha * delta_p
        p_new = torch.clamp(p_new, min=eps)
        p_new = p_new / p_new.sum(dim=1, keepdim=True)
        
        p_batch = p_new
    
    return p_batch

def newton_steady_state_optimized(model, W_batch, Y_batch, n_iter=10, eps=1e-12, tol=1e-8, reg=1e-6):
    """
    Optimized Newton's method with several improvements.
    """
    batch_size, n = W_batch.shape[0], model.n_nodes
    device = W_batch.device
    
    # Initialize
    p_batch = torch.full((batch_size, n), 1.0/n, device=device)
    
    # ===== KEY OPTIMIZATION 1: Pre-compute symmetric Y tensor =====
    # Since Jacobian needs Y[i,j,k] + Y[i,k,j], compute once outside loop
    Y_sym = Y_batch + Y_batch.transpose(1, 2)  # (batch, n, n, n)
    
    # ===== KEY OPTIMIZATION 2: Pre-allocate tensors =====
    outer = torch.empty(batch_size, n, n, device=device)
    J = torch.empty(batch_size, n, n, device=device)
    b = torch.empty(batch_size, n, device=device)
    
    # Pre-compute identity for regularization
    eye_reg = reg * torch.eye(n, device=device)
    
    for iter_idx in range(n_iter):
        # ===== Compute F(p) = W p + p Y p =====
        outer = p_batch.unsqueeze(2) * p_batch.unsqueeze(1)  # (batch, n, n)
        Q = torch.einsum("bijk,bjk->bi", Y_batch, outer)
        L = torch.bmm(W_batch, p_batch.unsqueeze(-1)).squeeze(-1)
        F = L + Q
        
        # ===== Early exit =====
        max_drift = torch.abs(F).max().item()
        if max_drift < tol:
            break
        
        # ===== KEY OPTIMIZATION: Single einsum for Jacobian =====
        J = W_batch + torch.einsum("bijk,bk->bij", Y_sym, p_batch) + eye_reg
        
        # ===== Constraint: Replace last row =====
        J[:, -1, :] = 1.0
        
        # ===== RHS =====
        b = -F.clone()
        b[:, -1] = 1.0 - p_batch.sum(dim=1)
        
        # ===== Solve =====
        try:
            delta_p = torch.linalg.solve(J, b)
        except RuntimeError:
            delta_p = torch.linalg.lstsq(J, b.unsqueeze(-1)).solution.squeeze(-1)
        
        # ===== Update (in-place where possible) =====
        p_batch = p_batch + delta_p
        p_batch.clamp_(min=eps)
        p_batch.div_(p_batch.sum(dim=1, keepdim=True))
    
    return p_batch

# def lbfgs_steady_state_batch(model, W_batch, Y_batch, max_iter=20, tol=1e-8):
#     """
#     Apply L-BFGS separately to each batch element.
#     More flexible but slightly slower.
#     """
#     batch_size, n = W_batch.shape[0], model.n_nodes
#     device = W_batch.device
    
#     p_results = []
    
#     for b in range(batch_size):
#         W_b = W_batch[b:b+1]  # Keep batch dimension
#         Y_b = Y_batch[b:b+1]
        
#         logits = torch.zeros(1, n, device=device, requires_grad=True)
        
#         optimizer = torch.optim.LBFGS(
#             [logits], 
#             max_iter=max_iter,
#             tolerance_grad=tol,
#             tolerance_change=tol,
#             line_search_fn='strong_wolfe'
#         )
        
#         def closure():
#             optimizer.zero_grad()
#             p = torch.softmax(logits, dim=1)
            
#             outer = p.unsqueeze(1) * p.unsqueeze(2)
#             Q = torch.einsum("bijk,bjk->bi", Y_b, outer)
#             L = torch.bmm(W_b, p.unsqueeze(-1)).squeeze(-1)
#             F = L + Q
            
#             loss = (F ** 2).sum()
#             loss.backward()
#             return loss
        
#         optimizer.step(closure)
        
#         with torch.no_grad():
#             p_results.append(torch.softmax(logits, dim=1))
    
#     return torch.cat(p_results, dim=0)
        
        
        # Get a single example from the training dataset
z_seq, labels, targets = train_loader.dataset[2000]

# Add batch dimension and move to device
z_seq = z_seq.unsqueeze(0).to(device)  # Shape: (1, N+1, D)
labels = labels.unsqueeze(0).to(device)  # Shape: (1, N)
targets = targets.unsqueeze(0).to(device).long() - 1  # Shape: (1,)

# print("Example input:")
# print(f"z_seq shape: {z_seq.shape}")
# print(f"labels shape: {labels.shape}")
# print(f"z_seq:\n{z_seq}")
# print(f"labels: {labels}")

# Flatten z_seq as the model expects
z_flat = z_seq.reshape(1, -1)  # Shape: (1, (N+1)*D)
# print(f"\nFlattened z shape: {z_flat.shape}")

# Compute the rate matrix W and rate tensor Y
model.eval()
with torch.no_grad():
    W_mat = model.compute_rate_matrix_W(z_flat)
    Y_tensor = model.compute_rate_tensor_Y(z_flat)

print(f"\nRate matrix W shape: {W_mat.shape}")
print(f"Rate matrix W:\n{W_mat[0]}")  # Print the first (and only) matrix in the batch

# Verify that columns sum to approximately zero (as they should for a rate matrix)
col_sums = W_mat[0].sum(dim=0)
# print(f"\nColumn sums of W (should be ~0): {col_sums}")

# print(f"\nRate tensor Y shape: {Y_tensor.shape}")
# print(f"Rate tensor Y[0] (first batch):\n{Y_tensor[0]}")

# Verify the constraint: Y[j,j,k] = -sum_{i≠j} Y[i,j,k]
# Check for j=0, k=0 as an example
j, k = 0, 0
sum_off_diag = Y_tensor[0, :, j, k].sum() - Y_tensor[0, j, j, k]  # Sum excluding diagonal
# print(f"\nConstraint check for Y[j={j},j={j},k={k}]:")
# print(f"  Y[{j},{j},{k}] = {Y_tensor[0, j, j, k].item():.6f}")
# print(f"  -Sum_{{i≠{j}}} Y[i,{j},{k}] = {-sum_off_diag.item():.6f}")
# print(f"  Difference (should be ~0): {(Y_tensor[0, j, j, k] + sum_off_diag).item():.6e}")

# Compute steady state using nonlinear solver
n_iter = 10
step_size = 0.05

start_time = time.time()
for _ in range(100):
    with torch.no_grad():
        #p_steady = model.direct_solve_steady_state(W_mat, Y_tensor, n_iter=n_iter, step_size=step_size)
        p_steady = model.newton_steady_state(W_mat, Y_tensor, n_iter=n_iter)
        #p_steady =  newton_steady_state_optimized(model, W_mat, Y_tensor, n_iter=n_iter, reg = 0)
        #p_steady = newton_steady_state(model,W_mat, Y_tensor, n_iter=n_iter)
        #p_steady = newton_steady_state(model, W_mat, Y_tensor, n_iter=n_iter)
        #p_steady = lbfgs_steady_state_batch(model, W_mat, Y_tensor, max_iter=n_iter, tol = )
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

print(f"\nSteady state distribution (π):")
print(f"Shape: {p_steady.shape}")
print(f"π = {p_steady[0]}")
print(f"Sum of π (should be 1.0): {p_steady[0].sum().item()}")

# Verify the steady state: W p + p Y p should be approximately 0
with torch.no_grad():
    # Linear term: W @ p
    linear_term = torch.einsum("bij,bj->bi", W_mat, p_steady)
    
    # Quadratic term: sum_{j,k} Y[i,j,k] * p_j * p_k
    outer = p_steady.unsqueeze(2) * p_steady.unsqueeze(1)  # (batch_size, n, n)
    quadratic_term = torch.einsum("bijk,bjk->bi", Y_tensor, outer)
    
    # Total should be ~0
    total_drift = linear_term + quadratic_term
    
print(f"\nSteady state verification (W p + p Y p):")
print(f"Linear term (W @ p): {linear_term[0]}")
print(f"Quadratic term (p Y p): {quadratic_term[0]}")
print(f"Total (should be ~0): {total_drift[0]}")
print(f"Max absolute drift: {torch.abs(total_drift[0]).max().item():.6e}")



Rate matrix W shape: torch.Size([1, 3, 3])
Rate matrix W:
tensor([[-2.7088e+02,  1.5343e+00,  4.2405e-01],
        [ 4.6324e-01, -1.5737e+00,  8.0553e-01],
        [ 2.7042e+02,  3.9341e-02, -1.2296e+00]])
Time taken: 0.17818021774291992 seconds

Steady state distribution (π):
Shape: torch.Size([1, 3])
π = tensor([0.0112, 0.0499, 0.9389])
Sum of π (should be 1.0): 1.0

Steady state verification (W p + p Y p):
Linear term (W @ p): tensor([-2.5631,  0.6831,  1.8801])
Quadratic term (p Y p): tensor([ 2.5631, -0.6831, -1.8801])
Total (should be ~0): tensor([0., 0., 0.])
Max absolute drift: 0.000000e+00
